In [1]:
import numpy as np
import pandas as pd
import os
import re
import pandasql as psql

In [2]:
state_abb_dict = {'Alabama': 'AL',
                  'Alaska': 'AK',
                  'Arizona': 'AZ',
                  'Arkansas': 'AR',
                  'California': 'CA',
                  'Colorado': 'CO',
                  'Connecticut': 'CT',
                  'Delaware': 'DE',
                  'DC': 'DC',
                  'Florida': 'FL',
                  'Georgia': 'GA',
                  'Hawaii': 'HI',
                  'Idaho': 'ID',
                  'Illinois': 'IL',
                  'Indiana': 'IN',
                  'Iowa': 'IA',
                  'Kansas': 'KS',
                  'Kentucky': 'KY',
                  'Louisiana': 'LA',
                  'Maine': 'ME',
                  'Maryland': 'MD',
                  'Massachusetts': 'MA',
                  'Michigan': 'MI',
                  'Minnesota': 'MN',
                  'Mississippi': 'MS',
                  'Missouri': 'MO',
                  'Montana': 'MT',
                  'Nebraska': 'NE',
                  'Nevada': 'NV',
                  'NewHampshire': 'NH',
                  'NewJersey': 'NJ',
                  'NewMexico': 'NM',
                  'NewYork': 'NY',
                  'NorthCarolina': 'NC',
                  'NorthDakota': 'ND',
                  'Ohio': 'OH',
                  'Oklahoma': 'OK',
                  'Oregon': 'OR',
                  'Pennsylvania': 'PA',
                  'RhodeIsland': 'RI',
                  'SouthCarolina': 'SC',
                  'SouthDakota': 'SD',
                  'Tennessee': 'TN',
                  'Texas': 'TX',
                  'Utah': 'UT',
                  'Vermont': 'VT',
                  'Virginia': 'VA',
                  'Washington': 'WA',
                  'WestVirginia': 'WV',
                  'Wisconsin': 'WI',
                  'Wyoming': 'WY'}

fips_dict = {'Alabama': '01',
              'Alaska': '02',
              'Arizona': '04',
              'Arkansas': '05',
              'California': '06',
              'Colorado': '08',
              'Connecticut': '09',
              'Delaware': '10',
              'DC': '11',
              'Florida': '12',
              'Georgia': '13',
              'Hawaii': '15',
              'Idaho': '16',
              'Illinois': '17',
              'Indiana': '18',
              'Iowa': '19',
              'Kansas': '20',
              'Kentucky': '21',
              'Louisiana': '22',
              'Maine': '23',
              'Maryland': '24',
              'Massachusetts': '25',
              'Michigan': '26',
              'Minnesota': '27',
              'Mississippi': '28',
              'Missouri': '29',
              'Montana': '30',
              'Nebraska': '31',
              'Nevada': '32',
              'NewHampshire': '33',
              'NewJersey': '34',
              'NewMexico': '35',
              'NewYork': '36',
              'NorthCarolina': '37',
              'NorthDakota': '38',
              'Ohio': '39',
              'Oklahoma': '40',
              'Oregon': '41',
              'Pennsylvania': '42',
              'RhodeIsland': '44',
              'SouthCarolina': '45',
              'SouthDakota': '46',
              'Tennessee': '47',
              'Texas': '48',
              'Utah': '49',
              'Vermont': '50',
              'Virginia': '51',
              'Washington': '53',
              'WestVirginia': '54',
              'Wisconsin': '55',
              'Wyoming': '56'}

state_dict = {'Alabama': 7,
              'Arizona': 9,
              'Arkansas': 4,
              'California': 53,
              'Colorado': 8,
              'Connecticut': 5,
              'Florida': 27,
              'Georgia': 14,
              'Hawaii': 2,
              'Idaho': 2,
              'Illinois': 18,
              'Indiana': 9,
              'Iowa': 4,
              'Kansas': 4,
              'Kentucky': 6,
              'Louisiana': 6,
              'Maine': 2,
              'Maryland': 8,
              'Massachusetts': 9,
              'Michigan': 14,
              'Minnesota': 8,
              'Mississippi': 4,
              'Missouri': 8,
              'Nebraska': 3,
              'Nevada': 4,
              'NewHampshire': 2,
              'NewJersey': 12,
              'NewMexico': 3,
              'NewYork': 27,
              'NorthCarolina': 13,
              'Ohio': 16,
              'Oklahoma': 5,
              'Oregon': 5,
              'Pennsylvania': 18,
              'RhodeIsland': 2,
              'SouthCarolina': 7,
              'Tennessee': 9,
              'Texas': 36,
              'Utah': 4,
              'Virginia': 11,
              'Washington': 10,
              'WestVirginia': 3,
              'Wisconsin': 8}

In [3]:
# Make an empty dataframe
def create_df():
    dtypes = np.dtype(
        [
            ('State', object),
            ('STATE_ABB', object),
            ('FIPS', object),
            ('CD', object),
            ('Median household income', object),
            ('Median nonfamily income', object),
            ('Persons aged 18 to 64 in poverty', object),
            ('Persons over 65 in poverty', object),
            ('Unemployed civilians', object),
            ('Less than 9th grade', object),
            ('9th to 12th grade, no diploma', object),
            ('High school graduate (includes equivalency)', object),
            ('Some college, no degree', object),
            ('Associates degree', object),
            ('Bachelors degree', object),
            ('Graduate or professional degree', object)
        ]
    )
    acs_final_df = pd.DataFrame(np.empty(0, dtype=dtypes))
    return acs_final_df

def new_arr(num_cd):
    districts = num_cd
    i = 0
    arr = []
    while i < (districts + 1) * 3 + 1:
        arr.append(i)
        i += 1
    return arr

def file_to_state(filename):
    text = filename
    new_text = text[0:10] + '(.*)' + text[-4:]
    result = re.search(new_text, text)
    return result.group(1)

In [4]:
# One CD state for CD dataset
my_dir = '/Users/darionguan/Desktop/Presidential Prediction/ACS/'
filename = 'ACSprofileAlaska.csv'
state_name = file_to_state(filename)
# acs_df = read_one_cd(my_dir + filename, state_name)

acs_df = pd.read_csv(my_dir + filename, header=None, usecols = [0, 1, 2, 3])

# Omit first three rows
acs_df = acs_df.iloc[3:,:]
# Change this value to 'Stat'
acs_df.iloc[1,0] = 'Stat'
# Reset index
acs_df = acs_df.reset_index(drop=True)

# Transpose
acs_df = acs_df.transpose()
# Strip trailing whitespace
acs_df.iloc[0] = acs_df.iloc[0].str.strip()
# First row becomes column names
acs_df.columns = acs_df.iloc[0]
# Drop first row
acs_df = acs_df.drop([0])
# Forward fill NaN on 'Subject' column
acs_df.loc[:,['Subject']] = acs_df.loc[:,['Subject']].ffill()
# If one CD, then change 'Subject' to '1'
acs_df.loc[acs_df['Subject'].str.startswith('Congressional District (at'), 'Subject'] = '1'.zfill(2)
# For DC, 'Subject' string starts with 'Delegate' as opposed to 'Congressional...'
acs_df.loc[acs_df['Subject'].str.startswith('Delegate'), 'Subject'] = '1'.zfill(2)
# Rename 'Subject' column to 'CD'
acs_df = acs_df.rename(columns={'Subject': 'CD'})

# Subset where Stat column is 'Pct'
pct_df = acs_df[acs_df['Stat'] == 'Pct']

# Subset where Stat column is 'Number'
acs_df = acs_df[acs_df['Stat'] == 'Number']

# Drop 'Stat' column
acs_df.drop('Stat', axis=1, inplace=True)
pct_df.drop('Stat', axis=1, inplace=True)

# Subset percentages
pct_df = pct_df[['CD', 'Persons aged 18 to 64 in poverty', 'Persons over 65 in poverty', 'Unemployed civilians']]

# Subset main df
acs_df = acs_df[['CD', 'Median household income', 'Median nonfamily income', 'Less than 9th grade',
                 '9th to 12th grade, no diploma', 'High school graduate (includes equivalency)',
                 'Some college, no degree', 'Associates degree', 'Bachelors degree',
                 'Graduate or professional degree']]

# Join columns from pct_df to acs_df
acs_df['Persons aged 18 to 64 in poverty'] = pct_df['Persons aged 18 to 64 in poverty'].values
acs_df['Persons over 65 in poverty'] = pct_df['Persons over 65 in poverty'].values
acs_df['Unemployed civilians'] = pct_df['Unemployed civilians'].values

# Set State column with default state name
acs_df['State'] = state_name

# Insert new column with state abbreviations based on state_abb_dict
acs_df.insert(0, 'STATE_ABB', acs_df['State'].map(state_abb_dict))    
# Insert column with FIPS state codes based on fips_dict
acs_df.insert(1, 'FIPS', acs_df['State'].map(fips_dict))
acs_df


,STATE_ABB,FIPS,CD,Median household income,Median nonfamily income,Less than 9th grade,"9th to 12th grade, no diploma",High school graduate (includes equivalency),"Some college, no degree",Associates degree,Bachelors degree,Graduate or professional degree,Persons aged 18 to 64 in poverty,Persons over 65 in poverty,Unemployed civilians,State
1,AK,02,01,75463,47508,10424,20666,139156,124261,43394,89583,56574,9.5,6.9,5.8,Alaska


In [5]:
# Multi- CD state for CD dataset

my_dir = '/Users/darionguan/Desktop/Presidential Prediction/ACS/'
filename = 'ACSprofileAlabama.csv'
state_name = file_to_state(filename)
cols = new_arr(state_dict[file_to_state(filename)])
state_name = file_to_state(filename)
# acs_df = read_multi_cd(my_dir + filename, state_name)

acs_df = pd.read_csv(my_dir + filename, header=None, usecols=cols)

# Omit first three rows
acs_df = acs_df.iloc[3:,:]
# Change this value to 'Stat'
acs_df.iloc[1,0] = 'Stat'
# Reset index
acs_df = acs_df.reset_index(drop=True)

# Transpose
acs_df = acs_df.transpose()
# Strip trailing whitespace
acs_df.iloc[0] = acs_df.iloc[0].str.strip()
# First row becomes column names
acs_df.columns = acs_df.iloc[0]
# Drop first row
acs_df = acs_df.drop([0])
# Forward fill NaN on 'Subject' column
acs_df.loc[:,['Subject']] = acs_df.loc[:,['Subject']].ffill()
# Substring 'Subject' values
acs_df['Subject'] = acs_df.Subject.mask(acs_df.Subject.str.len() > 24, acs_df.Subject.str[0:25])
# Strip trailing whitespace
acs_df['Subject'] = acs_df['Subject'].str.rstrip()
# Substring to last two characters if 'Subject' values start with 'Congressional'
acs_df['Subject'] = acs_df.Subject.mask(acs_df.Subject.str.startswith('Congressional'), acs_df.Subject.str[-2:])
# Strip leading whitespace
acs_df['Subject'] = acs_df['Subject'].str.lstrip()
# Fill in leading zeros
acs_df['Subject'] = acs_df['Subject'].astype(str).str.zfill(2)

# Rename 'Subject' column to 'CD'
acs_df = acs_df.rename(columns={'Subject': 'CD'})

# Subset where Stat column is 'Pct'
pct_df = acs_df[acs_df['Stat'] == 'Pct']

# Subset where Stat column is 'Number'
acs_df = acs_df[acs_df['Stat'] == 'Number']

# Drop 'Stat' column
acs_df.drop('Stat', axis=1, inplace=True)
pct_df.drop('Stat', axis=1, inplace=True)

# Subset percentages
pct_df = pct_df[['CD', 'Persons aged 18 to 64 in poverty', 'Persons over 65 in poverty', 'Unemployed civilians']]

# Subset main df
acs_df = acs_df[['CD', 'Median household income', 'Median nonfamily income', 'Less than 9th grade',
                 '9th to 12th grade, no diploma', 'High school graduate (includes equivalency)',
                 'Some college, no degree', 'Associates degree', 'Bachelors degree',
                 'Graduate or professional degree']]

# Join columns from pct_df to acs_df
acs_df['Persons aged 18 to 64 in poverty'] = pct_df['Persons aged 18 to 64 in poverty'].values
acs_df['Persons over 65 in poverty'] = pct_df['Persons over 65 in poverty'].values
acs_df['Unemployed civilians'] = pct_df['Unemployed civilians'].values

# Set State column with default state name
acs_df['State'] = state_name

# Insert new column with state abbreviations based on state_abb_dict
acs_df.insert(0, 'STATE_ABB', acs_df['State'].map(state_abb_dict))    
# Insert column with FIPS state codes based on fips_dict
acs_df.insert(1, 'FIPS', acs_df['State'].map(fips_dict))

# Remove the last row
acs_df = acs_df.iloc[:-1]
# Reset index
acs_df = acs_df.reset_index(drop=True)

acs_df

,STATE_ABB,FIPS,CD,Median household income,Median nonfamily income,Less than 9th grade,"9th to 12th grade, no diploma",High school graduate (includes equivalency),"Some college, no degree",Associates degree,Bachelors degree,Graduate or professional degree,Persons aged 18 to 64 in poverty,Persons over 65 in poverty,Unemployed civilians,State
0,AL,01,01,50663,30805,14362,44540,167287,103009,44489,80114,44217,14.4,10.2,5.7,Alabama
1,AL,01,02,50494,31799,19065,42762,147536,100874,43236,68419,40177,15.6,8.8,4.5,Alabama
2,AL,01,03,51925,25093,23068,49441,150179,99658,47236,65939,50452,15.4,10.2,5.9,Alabama
3,AL,01,04,47531,22794,24930,56731,163910,105245,43963,54939,29016,14.7,11.2,4.3,Alabama
4,AL,01,05,59950,33709,17397,36831,138163,101670,44494,106173,64018,12.8,7.9,4.3,Alabama
5,AL,01,06,69072,36650,14533,29426,114939,101328,38726,115867,70983,8.6,9.2,3.3,Alabama
6,AL,01,07,38023,24132,16965,43022,157227,88689,39770,56524,38519,22,16.7,6.8,Alabama


In [6]:
# Function for reading in 1 CD state for CD dataset
def read_one_cd(filename, state_name):
    acs_df = pd.read_csv(filename, header=None, usecols=[0, 1, 2, 3])

    # Omit first three rows
    acs_df = acs_df.iloc[3:,:]
    # Change this value to 'Stat'
    acs_df.iloc[1,0] = 'Stat'
    # Reset index
    acs_df = acs_df.reset_index(drop=True)

    # Transpose
    acs_df = acs_df.transpose()
    # Strip trailing whitespace
    acs_df.iloc[0] = acs_df.iloc[0].str.strip()
    # First row becomes column names
    acs_df.columns = acs_df.iloc[0]
    # Drop first row
    acs_df = acs_df.drop([0])
    # Forward fill NaN on 'Subject' column
    acs_df.loc[:,['Subject']] = acs_df.loc[:,['Subject']].ffill()
    # If one CD, then change 'Subject' to '1'
    acs_df.loc[acs_df['Subject'].str.startswith('Congressional District (at'), 'Subject'] = '1'.zfill(2)
    # For DC, 'Subject' string starts with 'Delegate' as opposed to 'Congressional...'
    acs_df.loc[acs_df['Subject'].str.startswith('Delegate'), 'Subject'] = '1'.zfill(2)
    # Fill in leading zeros
    acs_df['Subject'] = acs_df['Subject'].astype(str).str.zfill(2)

    # Rename 'Subject' column to 'CD'
    acs_df = acs_df.rename(columns={'Subject': 'CD'})
    
    # Prep subset df
    subset_df = acs_df[acs_df['Stat'] == 'Number']
    subset_df = subset_df[['Median household income', 'Median nonfamily income']]
    
    # Prep main df
    acs_df = acs_df[acs_df['Stat'] == 'Pct']
    acs_df = acs_df[['CD', 'Persons aged 18 to 64 in poverty', 'Persons over 65 in poverty', 'Unemployed civilians',
                     'Less than 9th grade', '9th to 12th grade, no diploma',
                     'High school graduate (includes equivalency)', 'Some college, no degree',
                     'Associates degree', 'Bachelors degree', 'Graduate or professional degree']]
    
    # Join columns from subset_df to acs_df
    acs_df['Median household income'] = subset_df['Median household income'].values
    acs_df['Median nonfamily income'] = subset_df['Median nonfamily income'].values

    # Set State column with default state name
    acs_df['State'] = state_name

    # Insert new column with state abbreviations based on state_abb_dict
    acs_df.insert(0, 'STATE_ABB', acs_df['State'].map(state_abb_dict))    
    # Insert column with FIPS state codes based on fips_dict
    acs_df.insert(1, 'FIPS', acs_df['State'].map(fips_dict))

    # Reset index
    acs_df = acs_df.reset_index(drop=True)

    return acs_df

In [7]:
# Function for reading multi-CD state for CD dataset
def read_multi_cd(filename, cols, state_name):
    acs_df = pd.read_csv(filename, header=None, usecols=cols)

    # Omit first three rows
    acs_df = acs_df.iloc[3:,:]
    # Change this value to 'Stat'
    acs_df.iloc[1,0] = 'Stat'
    # Reset index
    acs_df = acs_df.reset_index(drop=True)

    # Transpose
    acs_df = acs_df.transpose()
    # Strip trailing whitespace
    acs_df.iloc[0] = acs_df.iloc[0].str.strip()
    # First row becomes column names
    acs_df.columns = acs_df.iloc[0]
    # Drop first row
    acs_df = acs_df.drop([0])
    # Forward fill NaN on 'Subject' column
    acs_df.loc[:,['Subject']] = acs_df.loc[:,['Subject']].ffill()
    # Substring 'Subject' values
    acs_df['Subject'] = acs_df.Subject.mask(acs_df.Subject.str.len() > 24, acs_df.Subject.str[0:25])
    # Strip trailing whitespace
    acs_df['Subject'] = acs_df['Subject'].str.rstrip()
    # Substring to last two characters if 'Subject' values start with 'Congressional'
    acs_df['Subject'] = acs_df.Subject.mask(acs_df.Subject.str.startswith('Congressional'), acs_df.Subject.str[-2:])
    # Strip leading whitespace
    acs_df['Subject'] = acs_df['Subject'].str.lstrip()
    # Fill in leading zeros
    acs_df['Subject'] = acs_df['Subject'].astype(str).str.zfill(2)

    # Rename 'Subject' column to 'CD'
    acs_df = acs_df.rename(columns={'Subject': 'CD'})

    # Prep subset df
    subset_df = acs_df[acs_df['Stat'] == 'Number']
    subset_df = subset_df[['Median household income', 'Median nonfamily income']]
    
    # Prep main df
    acs_df = acs_df[acs_df['Stat'] == 'Pct']
    acs_df = acs_df[['CD', 'Persons aged 18 to 64 in poverty', 'Persons over 65 in poverty', 'Unemployed civilians',
                     'Less than 9th grade', '9th to 12th grade, no diploma',
                     'High school graduate (includes equivalency)', 'Some college, no degree',
                     'Associates degree', 'Bachelors degree', 'Graduate or professional degree']]
    
    # Join columns from subset_df to acs_df
    acs_df['Median household income'] = subset_df['Median household income'].values
    acs_df['Median nonfamily income'] = subset_df['Median nonfamily income'].values

    # Set State column with default state name
    acs_df['State'] = state_name

    # Insert new column with state abbreviations based on state_abb_dict
    acs_df.insert(0, 'STATE_ABB', acs_df['State'].map(state_abb_dict))    
    # Insert column with FIPS state codes based on fips_dict
    acs_df.insert(1, 'FIPS', acs_df['State'].map(fips_dict))

    # Remove the last row
    acs_df = acs_df.iloc[:-1]
    # Reset index
    acs_df = acs_df.reset_index(drop=True)

    return acs_df

In [8]:
def loop_cds():
    # Make an empty dataframe
    acs_final_df = create_df()
    
    # Directory path
    my_dir = '/Users/darionguan/Desktop/Presidential Prediction/ACS/'

    # For every file in lst
    for filename in os.listdir(my_dir):
        # If the file ends with name of state that has 1 CD
        # Last statement used to dodge .DS_Store file
        if (filename.endswith('Alaska.csv') or filename.endswith('Delaware.csv') or filename.endswith('DC.csv') \
        or filename.endswith('Montana.csv') or filename.endswith('NorthDakota.csv') \
        or filename.endswith('SouthDakota.csv') or filename.endswith('Vermont.csv') \
        or filename.endswith('Wyoming.csv')) and filename.startswith('.') is False:
            # state_name variable to be used in read_one_cd() function
            state_name = file_to_state(filename)
            # Use function for reading 1 CD state
            acs_df = read_one_cd(my_dir + filename, state_name)
            # Concatenate working acs_df to acs_final_df
            acs_final_df = pd.concat([acs_final_df, acs_df], ignore_index=True)

        # Else - statement used to dodge .DS_Store file
        elif filename.startswith('.') is False:
            # Use file_to_state() to find state name from filename
            # Use state name as key to find corresponding number of CDs in state_dict
            # Create an array to be used as number of cols read from csv
            cols = new_arr(state_dict[file_to_state(filename)])
            # state_name variable to be used in read_multi_cd() function
            state_name = file_to_state(filename)
            # Use function for reading multi-CD state
            acs_df = read_multi_cd(my_dir + filename, cols, state_name)
            # Concatenate working acs_df to acs_final_df
            acs_final_df = pd.concat([acs_final_df, acs_df], ignore_index=True)
            
    return acs_final_df

In [9]:
acs_final_df = loop_cds()
    
# Sort by the below columns
acs_final_df = acs_final_df.sort_values(by=['FIPS', 'CD'], ascending=[True, True])

# Reindex
acs_final_df = acs_final_df.reset_index(drop=True)

# Convert dtypes
acs_final_df = acs_final_df.astype({'Median household income': 'int64',
                                    'Median nonfamily income': 'int64',
                                    'Persons aged 18 to 64 in poverty': 'float64',
                                    'Persons over 65 in poverty': 'float64',
                                    'Unemployed civilians': 'float64',
                                    'Less than 9th grade': 'float64',
                                    '9th to 12th grade, no diploma': 'float64',
                                    'High school graduate (includes equivalency)': 'float64',
                                    'Some college, no degree': 'float64',
                                    'Associates degree': 'float64',
                                    'Bachelors degree': 'float64',
                                    'Graduate or professional degree': 'float64'})

# Rename columns
acs_final_df = acs_final_df.rename(columns={acs_final_df.columns[4]: 'MedianHouseholdIncome',
                                            acs_final_df.columns[5]: 'MedianNonfamilyIncome',
                                            acs_final_df.columns[6]: 'Pct18to64InPoverty',
                                            acs_final_df.columns[7]: 'PctOver65InPoverty',
                                            acs_final_df.columns[8]: 'PctUnemployedCivilians',
                                            acs_final_df.columns[9]: 'PctLessThan9thGrade',
                                            acs_final_df.columns[10]: 'Pct9thto12thGrade',
                                            acs_final_df.columns[11]: 'PctHSGrad',
                                            acs_final_df.columns[12]: 'PctSomeCollege',
                                            acs_final_df.columns[13]: 'PctAssociatesDeg',
                                            acs_final_df.columns[14]: 'PctBachelorsDeg',
                                            acs_final_df.columns[15]: 'PctGradDeg'})

# Convert percentages to decimal
acs_final_df['Pct18to64InPoverty'] = acs_final_df['Pct18to64InPoverty'] / 100
acs_final_df['PctOver65InPoverty'] = acs_final_df['PctOver65InPoverty'] / 100
acs_final_df['PctUnemployedCivilians'] = acs_final_df['PctUnemployedCivilians'] / 100
acs_final_df['PctLessThan9thGrade'] = acs_final_df['PctLessThan9thGrade'] / 100
acs_final_df['Pct9thto12thGrade'] = acs_final_df['Pct9thto12thGrade'] / 100
acs_final_df['PctHSGrad'] = acs_final_df['PctHSGrad'] / 100
acs_final_df['PctSomeCollege'] = acs_final_df['PctSomeCollege'] / 100
acs_final_df['PctAssociatesDeg'] = acs_final_df['PctAssociatesDeg'] / 100
acs_final_df['PctBachelorsDeg'] = acs_final_df['PctBachelorsDeg'] / 100
acs_final_df['PctGradDeg'] = acs_final_df['PctGradDeg'] / 100

# Convert df to csv
acs_final_df.to_csv('acs_cd_final.csv')

acs_final_df

,State,STATE_ABB,FIPS,CD,MedianHouseholdIncome,MedianNonfamilyIncome,Pct18to64InPoverty,PctOver65InPoverty,PctUnemployedCivilians,PctLessThan9thGrade,Pct9thto12thGrade,PctHSGrad,PctSomeCollege,PctAssociatesDeg,PctBachelorsDeg,PctGradDeg
0,Alabama,AL,01,01,50663,30805,0.144,0.102,0.057,0.029,0.089,0.336,0.207,0.089,0.161,0.089
1,Alabama,AL,01,02,50494,31799,0.156,0.088,0.045,0.041,0.093,0.319,0.218,0.094,0.148,0.087
2,Alabama,AL,01,03,51925,25093,0.154,0.102,0.059,0.047,0.102,0.309,0.205,0.097,0.136,0.104
3,Alabama,AL,01,04,47531,22794,0.147,0.112,0.043,0.052,0.119,0.342,0.220,0.092,0.115,0.061
4,Alabama,AL,01,05,59950,33709,0.128,0.079,0.043,0.034,0.072,0.272,0.200,0.087,0.209,0.126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,Wisconsin,WI,55,05,77386,44574,0.060,0.070,0.032,0.018,0.033,0.263,0.200,0.105,0.260,0.123
433,Wisconsin,WI,55,06,63251,40066,0.070,0.079,0.025,0.019,0.049,0.337,0.199,0.113,0.194,0.088
434,Wisconsin,WI,55,07,60706,33054,0.092,0.076,0.029,0.023,0.052,0.345,0.210,0.126,0.167,0.077
435,Wisconsin,WI,55,08,65346,38058,0.081,0.068,0.029,0.027,0.043,0.335,0.202,0.120,0.191,0.081


In [10]:
# Function for reading in 1 CD state for CD dataset
def read_one_state(filename, state_name):
    acs_df = pd.read_csv(filename, header=None, usecols=[0, 1, 2, 3])

    # Omit first three rows
    acs_df = acs_df.iloc[3:,:]
    # Change this value to 'Stat'
    acs_df.iloc[1,0] = 'Stat'
    # Reset index
    acs_df = acs_df.reset_index(drop=True)

    # Transpose
    acs_df = acs_df.transpose()
    # Strip trailing whitespace
    acs_df.iloc[0] = acs_df.iloc[0].str.strip()
    # First row becomes column names
    acs_df.columns = acs_df.iloc[0]
    # Drop first row
    acs_df = acs_df.drop([0])
    # Forward fill NaN on 'Subject' column
    acs_df.loc[:,['Subject']] = acs_df.loc[:,['Subject']].ffill()
    
#     # If one CD, then change 'Subject' to '1'
#     acs_df.loc[acs_df['Subject'].str.startswith('Congressional District (at'), 'Subject'] = '1'.zfill(2)
#     # For DC, 'Subject' string starts with 'Delegate' as opposed to 'Congressional...'
#     acs_df.loc[acs_df['Subject'].str.startswith('Delegate'), 'Subject'] = '1'.zfill(2)
#     # Fill in leading zeros
#     acs_df['Subject'] = acs_df['Subject'].astype(str).str.zfill(2)

    # Change all rows of 'Subject' column to state_name
    acs_df['Subject'] = state_name

    # Rename 'Subject' column to 'State'
    acs_df = acs_df.rename(columns={'Subject': 'State'})
    
    # Prep subset df
    subset_df = acs_df[acs_df['Stat'] == 'Number']
    subset_df = subset_df[['Median household income', 'Median nonfamily income']]
    
    # Prep main df
    acs_df = acs_df[acs_df['Stat'] == 'Pct']
    acs_df = acs_df[['State', 'Persons aged 18 to 64 in poverty', 'Persons over 65 in poverty', 'Unemployed civilians',
                     'Less than 9th grade', '9th to 12th grade, no diploma',
                     'High school graduate (includes equivalency)', 'Some college, no degree',
                     'Associates degree', 'Bachelors degree', 'Graduate or professional degree']]
    
    # Join columns from subset_df to acs_df
    acs_df['Median household income'] = subset_df['Median household income'].values
    acs_df['Median nonfamily income'] = subset_df['Median nonfamily income'].values

    # Insert new column with state abbreviations based on state_abb_dict
    acs_df.insert(0, 'STATE_ABB', acs_df['State'].map(state_abb_dict))    
    # Insert column with FIPS state codes based on fips_dict
    acs_df.insert(1, 'FIPS', acs_df['State'].map(fips_dict))

    # Reset index
    acs_df = acs_df.reset_index(drop=True)

    return acs_df

In [11]:
# Function for reading multi-CD state for state dataset
def read_multi_state(filename, cols, state_name):
    acs_df = pd.read_csv(filename, header=None, usecols=cols)

    # Omit first three rows
    acs_df = acs_df.iloc[3:,:]
    # Change this value to 'Stat'
    acs_df.iloc[1,0] = 'Stat'
    # Reset index
    acs_df = acs_df.reset_index(drop=True)

    # Transpose
    acs_df = acs_df.transpose()
    # Strip trailing whitespace
    acs_df.iloc[0] = acs_df.iloc[0].str.strip()
    # First row becomes column names
    acs_df.columns = acs_df.iloc[0]
    # Drop first row
    acs_df = acs_df.drop([0])
    # Forward fill NaN on 'Subject' column
    acs_df.loc[:,['Subject']] = acs_df.loc[:,['Subject']].ffill()
    
    # Keep last three rows
    acs_df = acs_df[-3:]

    # Change all rows of 'Subject' column to state_name
    acs_df['Subject'] = state_name

    # Rename 'Subject' column to 'State'
    acs_df = acs_df.rename(columns={'Subject': 'State'})

    # Prep subset df
    subset_df = acs_df[acs_df['Stat'] == 'Number']
    subset_df = subset_df[['Median household income', 'Median nonfamily income']]
    
    # Prep main df
    acs_df = acs_df[acs_df['Stat'] == 'Pct']
    acs_df = acs_df[['State', 'Persons aged 18 to 64 in poverty', 'Persons over 65 in poverty', 'Unemployed civilians',
                     'Less than 9th grade', '9th to 12th grade, no diploma',
                     'High school graduate (includes equivalency)', 'Some college, no degree',
                     'Associates degree', 'Bachelors degree', 'Graduate or professional degree']]
    
    # Join columns from subset_df to acs_df
    acs_df['Median household income'] = subset_df['Median household income'].values
    acs_df['Median nonfamily income'] = subset_df['Median nonfamily income'].values

    # Insert new column with state abbreviations based on state_abb_dict
    acs_df.insert(0, 'STATE_ABB', acs_df['State'].map(state_abb_dict))    
    # Insert column with FIPS state codes based on fips_dict
    acs_df.insert(1, 'FIPS', acs_df['State'].map(fips_dict))

    # Reset index
    acs_df = acs_df.reset_index(drop=True)

    return acs_df

In [12]:
def loop_states():
    # Make an empty dataframe
    acs_final_df = create_df()
    
    # Directory path
    my_dir = '/Users/darionguan/Desktop/Presidential Prediction/ACS/'
    
    # For every file in lst
    for filename in os.listdir(my_dir):
        # If the file ends with name of state that has 1 CD
        # Last statement used to dodge .DS_Store file
        if (filename.endswith('Alaska.csv') or filename.endswith('Delaware.csv') or filename.endswith('DC.csv') \
        or filename.endswith('Montana.csv') or filename.endswith('NorthDakota.csv') \
        or filename.endswith('SouthDakota.csv') or filename.endswith('Vermont.csv') \
        or filename.endswith('Wyoming.csv')) and filename.startswith('.') is False:
            # state_name variable to be used in read_one_cd() function
            state_name = file_to_state(filename)
            # Use function for reading 1 CD state
            acs_df = read_one_state(my_dir + filename, state_name)
            # Concatenate working acs_df to acs_final_df
            acs_final_df = pd.concat([acs_final_df, acs_df], ignore_index=True)

        # Else - statement used to dodge .DS_Store file
        elif filename.startswith('.') is False:
            # Use file_to_state() to find state name from filename
            # Use state name as key to find corresponding number of CDs in state_dict
            # Create an array to be used as number of cols read from csv
            cols = new_arr(state_dict[file_to_state(filename)])
            # state_name variable to be used in read_multi_cd() function
            state_name = file_to_state(filename)
            # Use function for reading multi-CD state
            acs_df = read_multi_state(my_dir + filename, cols, state_name)
            # Concatenate working acs_df to acs_final_df
            acs_final_df = pd.concat([acs_final_df, acs_df], ignore_index=True)
    return acs_final_df

In [13]:
# Make an empty dataframe
acs_final_df = create_df()

# Directory path
my_dir = '/Users/darionguan/Desktop/Presidential Prediction/ACS/'

# # For every file in lst
# for filename in os.listdir(my_dir):
#     # If the file ends with name of state that has 1 CD
#     # Last statement used to dodge .DS_Store file
#     if (filename.endswith('Alaska.csv') or filename.endswith('Delaware.csv') or filename.endswith('DC.csv') \
#     or filename.endswith('Montana.csv') or filename.endswith('NorthDakota.csv') \
#     or filename.endswith('SouthDakota.csv') or filename.endswith('Vermont.csv') \
#     or filename.endswith('Wyoming.csv')) and filename.startswith('.') is False:
#         # state_name variable to be used in read_one_cd() function
#         state_name = file_to_state(filename)
#         # Use function for reading 1 CD state
#         acs_df = read_one_state(my_dir + filename, state_name)
#         # Concatenate working acs_df to acs_final_df
#         acs_final_df = pd.concat([acs_final_df, acs_df], ignore_index=True)
        
#     # Else - statement used to dodge .DS_Store file
#     elif filename.startswith('.') is False:
#         # Use file_to_state() to find state name from filename
#         # Use state name as key to find corresponding number of CDs in state_dict
#         # Create an array to be used as number of cols read from csv
#         cols = new_arr(state_dict[file_to_state(filename)])
#         # state_name variable to be used in read_multi_cd() function
#         state_name = file_to_state(filename)
#         # Use function for reading multi-CD state
#         acs_df = read_multi_state(my_dir + filename, cols, state_name)
#         # Concatenate working acs_df to acs_final_df
#         acs_final_df = pd.concat([acs_final_df, acs_df], ignore_index=True)

acs_final_df = loop_states()

# Sort by columns
acs_final_df = acs_final_df.sort_values(by='FIPS', ascending=True)

# Drop CD column
acs_final_df = acs_final_df.drop('CD', axis=1)

# Reindex
acs_final_df = acs_final_df.reset_index(drop=True)

# Convert dtypes
acs_final_df = acs_final_df.astype({'Median household income': 'int64',
                                    'Median nonfamily income': 'int64',
                                    'Persons aged 18 to 64 in poverty': 'float64',
                                    'Persons over 65 in poverty': 'float64',
                                    'Unemployed civilians': 'float64',
                                    'Less than 9th grade': 'float64',
                                    '9th to 12th grade, no diploma': 'float64',
                                    'High school graduate (includes equivalency)': 'float64',
                                    'Some college, no degree': 'float64',
                                    'Associates degree': 'float64',
                                    'Bachelors degree': 'float64',
                                    'Graduate or professional degree': 'float64'})

# Rename columns
acs_final_df = acs_final_df.rename(columns={acs_final_df.columns[3]: 'MedianHouseholdIncome',
                                            acs_final_df.columns[4]: 'MedianNonfamilyIncome',
                                            acs_final_df.columns[5]: 'Pct18to64InPoverty',
                                            acs_final_df.columns[6]: 'PctOver65InPoverty',
                                            acs_final_df.columns[7]: 'PctUnemployedCivilians',
                                            acs_final_df.columns[8]: 'PctLessThan9thGrade',
                                            acs_final_df.columns[9]: 'Pct9thto12thGrade',
                                            acs_final_df.columns[10]: 'PctHSGrad',
                                            acs_final_df.columns[11]: 'PctSomeCollege',
                                            acs_final_df.columns[12]: 'PctAssociatesDeg',
                                            acs_final_df.columns[13]: 'PctBachelorsDeg',
                                            acs_final_df.columns[14]: 'PctGradDeg'})

# Convert percentages to decimal
acs_final_df['Pct18to64InPoverty'] = acs_final_df['Pct18to64InPoverty'] / 100
acs_final_df['PctOver65InPoverty'] = acs_final_df['PctOver65InPoverty'] / 100
acs_final_df['PctUnemployedCivilians'] = acs_final_df['PctUnemployedCivilians'] / 100
acs_final_df['PctLessThan9thGrade'] = acs_final_df['PctLessThan9thGrade'] / 100
acs_final_df['Pct9thto12thGrade'] = acs_final_df['Pct9thto12thGrade'] / 100
acs_final_df['PctHSGrad'] = acs_final_df['PctHSGrad'] / 100
acs_final_df['PctSomeCollege'] = acs_final_df['PctSomeCollege'] / 100
acs_final_df['PctAssociatesDeg'] = acs_final_df['PctAssociatesDeg'] / 100
acs_final_df['PctBachelorsDeg'] = acs_final_df['PctBachelorsDeg'] / 100
acs_final_df['PctGradDeg'] = acs_final_df['PctGradDeg'] / 100

# Convert df to csv
acs_final_df.to_csv('acs_state_final.csv')

acs_final_df

,State,STATE_ABB,FIPS,MedianHouseholdIncome,MedianNonfamilyIncome,Pct18to64InPoverty,PctOver65InPoverty,PctUnemployedCivilians,PctLessThan9thGrade,Pct9thto12thGrade,PctHSGrad,PctSomeCollege,PctAssociatesDeg,PctBachelorsDeg,PctGradDeg
0,Alabama,AL,01,51734,29458,0.147,0.105,0.049,0.039,0.090,0.309,0.208,0.090,0.163,0.100
1,Alaska,AK,02,75463,47508,0.095,0.069,0.058,0.022,0.043,0.287,0.257,0.090,0.185,0.117
2,Arizona,AZ,04,62055,40198,0.127,0.090,0.051,0.051,0.073,0.237,0.250,0.087,0.188,0.113
3,Arkansas,AR,05,48952,27607,0.155,0.105,0.048,0.046,0.079,0.349,0.218,0.075,0.151,0.083
4,California,CA,06,80440,51676,0.107,0.105,0.051,0.087,0.073,0.206,0.206,0.079,0.219,0.131
5,Colorado,CO,08,77127,50092,0.093,0.072,0.037,0.032,0.044,0.210,0.203,0.084,0.266,0.160
6,Connecticut,CT,09,78833,45001,0.094,0.073,0.053,0.040,0.053,0.268,0.165,0.076,0.220,0.178
7,Delaware,DE,10,70176,44071,0.108,0.073,0.046,0.035,0.062,0.302,0.186,0.083,0.195,0.137
8,DC,DC,11,92266,76529,0.120,0.133,0.063,0.033,0.049,0.158,0.130,0.033,0.257,0.340
9,Florida,FL,12,59227,37292,0.117,0.107,0.045,0.046,0.070,0.284,0.194,0.099,0.193,0.114
